In [43]:
import dask
import sys
sys.path.append('../cli_common/')
import utils as cli_utils
import histomicstk.preprocessing.color_normalization as htk_cnorm
import large_image
import histomicstk.utils as htk_utils
from skimage.measure import regionprops
from skimage.segmentation import slic
import numpy as np
import h5py
from ctk_cli import CLIArgumentParser
import dask.distributed as dd
import logging
logging.basicConfig(level=logging.CRITICAL)
import histomicstk.segmentation as htk_seg

In [2]:
slidePath = 'TCGA.svs'
ts = large_image.getTileSource(slidePath)
# compute colorspace statistics (mean, variance) for whole slide
wsi_mean, wsi_stddev = htk_cnorm.reinhard_stats(slidePath, 0.1, 10)

INFO:girder:Using python for large_image caching


Using python for large_image caching


In [3]:
from skimage.measure import regionprops
from skimage.segmentation import slic

In [4]:
def compute_superpixel_data(img_path, tile_position, wsi_mean, wsi_stddev):
    
    # get slide tile source
    ts = large_image.getTileSource(img_path)

    # get requested tile information
    tile_info = ts.getSingleTile(
        tile_position=tile_position,
        resample=True,
        format=large_image.tilesource.TILE_FORMAT_NUMPY)
    
    
    im_tile = tile_info['tile'][:, :, :3]

    # get global x and y positions
    left = tile_info['gx']
    top = tile_info['gy']

    # get scale
    scale = tile_info['gwidth'] / tile_info['width']

    reference_mu_lab = [8.63234435, -0.11501964, 0.03868433]
    reference_std_lab = [0.57506023, 0.10403329, 0.01364062]
    
    # perform color normalization
    im_nmzd = htk_cnorm.reinhard(im_tile,
                                 reference_mu_lab, reference_std_lab,
                                 wsi_mean, wsi_stddev)
    patchSize = 32
    # compute the number of super-pixels
    im_width, im_height = im_nmzd.shape[:2]
    n_superpixels = (im_width/patchSize) * (im_height/patchSize)

    #
    # Generate labels using a superpixel algorithm (SLIC)
    # In SLIC, compactness controls image space proximity.
    # Higher compactness will make the shape of superpixels more square.
    #
    
    compactness = 50
    im_label = slic(im_nmzd, n_segments=n_superpixels,compactness=compactness) + 1

    region_props = regionprops(im_label)

    # set superpixel data list
    # set superpixel data list
    s_data = []
    x_cent = []
    y_cent = []
    x_brs = []
    y_brs = []

    for i in range(len(region_props)):
        # get x, y centroids for superpixel
        cen_x, cen_y = region_props[i].centroid

        # get bounds of superpixel region
        min_row, max_row, min_col, max_col = \
            get_patch_bounds(cen_x, cen_y, patchSize, im_width, im_height)

        # grab superpixel label mask
        lmask = (im_label[:, :] == region_props[i].label).astype(np.bool)

        # embed with center pixel in middle of padded window
        emask = np.zeros((lmask.shape[0] + 2, lmask.shape[1] + 2), dtype=np.bool)
        emask[1:-1, 1:-1] = lmask

        # find boundaries
        bx, by = htk_seg.label.trace_object_boundaries(emask)

        with np.errstate(invalid='ignore'):
            # remove redundant points
            mby, mbx = htk_utils.merge_colinear(by[0].astype(float), bx[0].astype(float))

        scaled_x = (mbx - 1) * scale
        scaled_y = (mby - 1) * scale

        # get superpixel boundary at highest-res
        x_brs.append(scaled_x + top)
        y_brs.append(scaled_y + left)

        
        rgb_data = im_nmzd[min_row:max_row, min_col:max_col]

        s_data.append(rgb_data)
        
        # get superpixel centers at highest-res
        x_cent.append(
            round((cen_x * scale + top), 1))
        y_cent.append(
            round((cen_y * scale + left), 1))

    return s_data, x_cent, y_cent, x_brs, y_brs

def get_patch_bounds(cx, cy, patch_size, m, n):

    half_patch_size = patch_size/2.0

    min_row = int(round(cx) - half_patch_size)
    max_row = int(round(cx) + half_patch_size)
    min_col = int(round(cy) - half_patch_size)
    max_col = int(round(cy) + half_patch_size)

    if min_row < 0:
        max_row = max_row - min_row
        min_row = 0

    if max_row > m-1:
        min_row = min_row - (max_row - (m-1))
        max_row = m-1

    if min_col < 0:
        max_col = max_col - min_col
        min_col = 0

    if max_col > n-1:
        min_col = min_col - (max_col - (n-1))
        max_col = n-1

    return min_row, max_row, min_col, max_col

In [5]:
slidePath = 'TCGA.svs'
ts = large_image.getTileSource(slidePath)
# compute colorspace statistics (mean, variance) for whole slide
wsi_mean, wsi_stddev = htk_cnorm.reinhard_stats(slidePath, 0.1, 10)

In [6]:
# compute tissue/foreground mask at low-res for whole slide images
im_fgnd_mask_lres, fgnd_seg_scale = cli_utils.segment_wsi_foreground_at_low_res(ts)

In [7]:
# compute foreground fraction of tiles in parallel using Dask
analysis_tile_size = 2048
analysis_mag = 10
it_kwargs = {
    'tile_size': {'width': analysis_tile_size},
    'scale': {'magnification': analysis_mag},
}

inputSlidePath = 0

tile_fgnd_frac_list = htk_utils.compute_tile_foreground_fraction(
    slidePath, im_fgnd_mask_lres, fgnd_seg_scale,
    **it_kwargs
)

print('\n>> Detecting superpixel data ...\n')


>> Detecting superpixel data ...



In [13]:
tile_result_list = []
min_fgnd_frac = 0.001


for tile in ts.tileIterator(**it_kwargs):
    tile_position = tile['tile_position']['position']
    if tile_fgnd_frac_list[tile_position] <= min_fgnd_frac:
        continue

    # detect superpixel data
    cur_result = compute_superpixel_data(slidePath,tile_position,wsi_mean, wsi_stddev)

    # append result to list
    tile_result_list.append(cur_result)

#tile_result_list = tile_result_list.compute()

# initiate output data list


In [16]:
superpixel_data = []
x_centroids = []
y_centroids = []
x_boundaries = []
y_boundaries = []

for s_data, x_cent, y_cent, x_brs, y_brs in tile_result_list:
        for s_d in s_data:
            superpixel_data.append(s_d)

        for x_c in x_cent:
            x_centroids.append(x_c)

        for y_c in y_cent:
            y_centroids.append(y_c)

        for x_b in x_brs:
            x_boundaries.append(x_b)

        for y_b in y_brs:
            y_boundaries.append(y_b)

superpixel_data = np.asarray(superpixel_data, dtype=np.float32)

n_superpixels = len(superpixel_data)
x_centroids = np.asarray(x_centroids).reshape((n_superpixels, 1))
y_centroids = np.asarray(y_centroids).reshape((n_superpixels, 1))

#
# Last, we can store the data
#
print('>> Writing superpixel data information')

output = h5py.File('spixelfeatures', 'w')
output.create_dataset('features', data=superpixel_data)
output.create_dataset('x_centroid', data=x_centroids)
output.create_dataset('y_centroid', data=y_centroids)
output.close()

>> Writing superpixel data information


In [17]:
superpixel_data

array([[[[249., 213., 242.],
         [249., 213., 242.],
         [249., 213., 242.],
         ...,
         [249., 213., 240.],
         [249., 213., 242.],
         [249., 213., 243.]],

        [[249., 213., 242.],
         [249., 213., 242.],
         [249., 213., 242.],
         ...,
         [248., 212., 240.],
         [248., 212., 241.],
         [249., 212., 243.]],

        [[249., 213., 242.],
         [249., 213., 242.],
         [249., 213., 242.],
         ...,
         [248., 212., 240.],
         [248., 212., 240.],
         [249., 212., 242.]],

        ...,

        [[249., 213., 241.],
         [249., 213., 243.],
         [249., 213., 243.],
         ...,
         [249., 213., 242.],
         [249., 213., 242.],
         [247., 212., 242.]],

        [[250., 213., 242.],
         [249., 213., 242.],
         [248., 213., 241.],
         ...,
         [249., 213., 242.],
         [249., 213., 243.],
         [247., 213., 244.]],

        [[247., 212., 242.],
       

In [18]:
print('>> Writing text boundary file')

boundary_file = open('BoundariesFile', 'w')

for i in range(n_superpixels):
    boundary_file.write("%.1f\t" % y_centroids[i, 0])
    boundary_file.write("%.1f\t" % x_centroids[i, 0])

for j in range(len(x_boundaries[i])):
    boundary_file.write("%d,%d " % (y_boundaries[i][j], x_boundaries[i][j]))
    boundary_file.write("\n")

>> Writing text boundary file


In [21]:
import h5py

In [22]:
f=h5py.File('spixelfeatures','r')
dset=f.keys()

In [25]:
dset[0], dset[1], dset[2]

(u'features', u'x_centroid', u'y_centroid')

In [26]:
### reading features daaset
import numpy as np
n1=f.get('features')
n1=np.array(n1)
n1.shape

(2891, 32, 32, 3)

In [30]:
### reading x_centroid daaset
import numpy as np
n2=f.get('x_centroid')
n2=np.array(n2)
n2.shape

(2891, 1)

In [31]:
### reading y_centroid dataset
import numpy as np
n3=f.get('y_centroid')
n3=np.array(n2)
n3.shape

(2891, 1)

In [32]:
### reading features dataset n1,
print n1[1]

[[[250. 213. 242.]
  [249. 213. 241.]
  [249. 212. 241.]
  ...
  [248. 212. 240.]
  [248. 212. 242.]
  [248. 212. 242.]]

 [[250. 213. 243.]
  [249. 213. 243.]
  [249. 213. 243.]
  ...
  [249. 212. 241.]
  [249. 212. 243.]
  [248. 212. 243.]]

 [[250. 213. 243.]
  [249. 213. 243.]
  [249. 214. 242.]
  ...
  [249. 212. 242.]
  [249. 212. 244.]
  [249. 212. 244.]]

 ...

 [[249. 213. 242.]
  [249. 213. 242.]
  [249. 213. 242.]
  ...
  [249. 213. 241.]
  [249. 213. 241.]
  [249. 213. 242.]]

 [[249. 212. 244.]
  [250. 212. 244.]
  [250. 214. 244.]
  ...
  [249. 213. 241.]
  [249. 213. 241.]
  [249. 213. 242.]]

 [[248. 213. 242.]
  [250. 212. 241.]
  [249. 212. 241.]
  ...
  [249. 213. 242.]
  [249. 213. 242.]
  [248. 213. 242.]]]


In [33]:
print n1[2]

[[[248. 212. 242.]
  [248. 212. 242.]
  [248. 212. 242.]
  ...
  [249. 213. 241.]
  [249. 213. 239.]
  [249. 213. 242.]]

 [[247. 212. 242.]
  [247. 212. 243.]
  [248. 212. 242.]
  ...
  [249. 213. 242.]
  [249. 213. 239.]
  [248. 213. 241.]]

 [[248. 212. 243.]
  [248. 212. 244.]
  [248. 212. 242.]
  ...
  [249. 213. 242.]
  [249. 213. 241.]
  [248. 213. 241.]]

 ...

 [[248. 212. 242.]
  [248. 211. 242.]
  [248. 213. 243.]
  ...
  [249. 213. 243.]
  [249. 213. 243.]
  [250. 213. 242.]]

 [[248. 212. 242.]
  [248. 211. 243.]
  [248. 211. 242.]
  ...
  [247. 212. 243.]
  [247. 212. 243.]
  [249. 213. 243.]]

 [[249. 213. 242.]
  [249. 213. 242.]
  [248. 213. 241.]
  ...
  [249. 213. 240.]
  [248. 213. 241.]
  [248. 213. 240.]]]


In [34]:
print(n2[0])

[17.]


In [36]:
### x_centroid coordinates
print(n2[:20])

[[17. ]
 [17. ]
 [17. ]
 [17. ]
 [17. ]
 [17. ]
 [17. ]
 [51.5]
 [51.5]
 [51.5]
 [51.5]
 [51.5]
 [51.5]
 [51.5]
 [85.5]
 [85.5]
 [85.5]
 [85.5]
 [85.5]
 [85.5]]


In [42]:
fb=open('BoundariesFile','r')
for line in fb: 
    print line

497.0	17.0	531.5	17.0	565.5	17.0	599.5	17.0	633.5	17.0	667.5	17.0	702.0	17.0	497.0	51.5	531.5	51.5	565.5	51.5	599.5	51.5	633.5	51.5	667.5	51.5	702.0	51.5	497.0	85.5	531.5	85.5	565.5	85.5	599.5	85.5	633.5	85.5	667.5	85.5	702.0	85.5	497.0	119.5	531.5	119.5	565.5	119.5	599.5	119.5	633.5	119.5	667.5	119.5	702.0	119.5	497.0	153.5	531.5	153.5	565.5	153.5	599.5	153.5	633.5	153.5	667.5	153.5	702.0	153.5	497.0	187.5	531.5	187.5	565.5	187.5	599.5	187.5	633.5	187.5	667.5	187.5	702.0	187.5	497.0	222.0	531.5	222.0	565.5	222.0	599.5	222.0	633.5	222.0	667.5	222.0	702.0	222.0	737.0	17.0	771.5	17.0	805.5	17.0	839.5	17.0	873.5	17.0	907.5	17.0	942.0	17.0	737.0	51.5	771.5	51.5	805.5	51.5	839.5	51.5	873.5	51.5	907.5	51.5	942.0	51.5	737.0	85.5	771.5	85.5	805.5	85.5	839.5	85.5	873.5	85.5	907.5	85.5	942.0	85.5	737.0	119.5	771.5	119.5	805.5	119.5	839.5	119.5	873.5	119.5	907.5	119.5	942.0	119.5	737.0	153.5	771.5	153.5	805.5	153.5	839.5	153.5	873.5	153.5	907.5	153.5	942.0	153.5	737.0	187.5	771.5	187.5	805.5	187.